You attach an electronic nanomesh tattoo to the girl’s skin.

In order to get correct readings from the interface between her skin and your electronics, you’ll need to descramble the data from all 14 channels.

Each channel is a data stream that uses a big-endian 8-bit encoding. Every byte in the channel points to another byte in the stream, i.e. the value of the byte is the offset in bytes from the beginning.

The channels can contain invalid bytes, which means following their pointer would cause an overflow off the stream.

Start reading data from the beginning of the channel, and ignore any invalid bytes until you encounter the first valid byte. After this, follow valid byte pointers until you reach another invalid byte.

This invalid byte contributes one character to the password. Repeat this process for all 14 channels to read the complete password.

In [1]:
#!fsharp
let input = System.IO.File.ReadAllLines "./tattoo_input.txt" |> List.ofArray

let rec parseByte (acc: int) (chars: char list) =
    match chars with
    | [] -> acc
    | '0' :: rest -> parseByte acc rest
    | '1' :: rest -> parseByte (acc + (1 <<< (8 - List.length chars))) rest
    | c -> failwith "Unknown input"

let parseInputString (s: string) = Seq.chunkBySize 8 s |> Seq.map (List.ofArray >> List.rev >> parseByte 0)

let inputBytes = List.map (parseInputString >> Array.ofSeq) input

let rec findPasswordChar (ix: int) (foundFirstValid: bool) (arr: int array) =
    match Array.tryItem ix arr, foundFirstValid with
    | None, false -> findPasswordChar (ix + 1) false arr
    | Some x, false -> if x < Array.length arr then findPasswordChar x true arr else findPasswordChar (ix + 1) false arr
    | Some x, true -> findPasswordChar x true arr
    | None, true -> ix

let ascii = System.Text.ASCIIEncoding ()

let foundBytes = List.map (findPasswordChar 0 false >> byte) inputBytes |> Array.ofList

ascii.GetChars foundBytes |> System.String

left-ventricle